In [20]:
import shutil, os
os.makedirs("data/raw", exist_ok=True)   
orig = r"C:\Users\Admin\OneDrive\Documents\USIU\Datawarehousing\Groupwork\data\raw\sales_and_customer_insights.xlsx"
copied = "data/raw/sales_and_customer_insights_copy.xlsx"
shutil.copyfile(orig, copied)
print("Copied raw file to:", copied)

# Adding missing values to dataset
import pandas as pd
import numpy as np

# List available sheets first
xls = pd.ExcelFile("data/raw/sales_and_customer_insights_copy.xlsx")
print("Available sheets:", xls.sheet_names)

# Load just the first few rows of the first sheet
df = pd.read_excel("data/raw/sales_and_customer_insights_copy.xlsx")
df_preview = pd.read_excel(xls, sheet_name=0, nrows=10)
print("Shape:", df_preview.shape)
df_preview.head()

import numpy as np
import pandas as pd

# Assuming df is already loaded (the full Online Retail dataset)
print("Starting shape:", df.shape)

# ---- Step 1: Introduce missingness (vectorized, 2%) ----
def introduce_missingness_large(df, frac=0.02, seed=42):
    np.random.seed(seed)
    dfm = df.copy()
    n = len(dfm)
    for col in dfm.columns:
        # pick random indices for each column to make NaN
        k = int(n * frac)
        idx = np.random.choice(dfm.index, k, replace=False)
        dfm.loc[idx, col] = np.nan
    return dfm

df_dirty = introduce_missingness_large(df, frac=0.02)
print("Added missing values ✅")

# ---- Step 2: Introduce duplicates (1%) ----
def introduce_duplicates_large(df, frac_rows=0.01, seed=1):
    np.random.seed(seed)
    ndup = int(len(df) * frac_rows)
    sample = df.sample(ndup, replace=False, random_state=seed)
    df_dup = pd.concat([df, sample], ignore_index=True)
    return df_dup

df_dirty = introduce_duplicates_large(df_dirty, frac_rows=0.01)
print("Added duplicates ✅")

# ---- Step 3: Introduce weird date formats (3%) ----
def introduce_weird_dates_large(df, date_col="InvoiceDate", frac=0.03, seed=123):
    df2 = df.copy()
    if date_col not in df2.columns:
        return df2
    np.random.seed(seed)
    idx = df2.sample(frac=frac, random_state=seed).index
    for i in idx:
        val = df2.at[i, date_col]
        if pd.isna(val): 
            continue
        try:
            dt = pd.to_datetime(val)
            df2.at[i, date_col] = dt.strftime("%d-%b-%Y")
        except Exception:
            pass
    return df2

df_dirty = introduce_weird_dates_large(df_dirty, "InvoiceDate")
print("Added weird date formats ✅")

# ---- Step 4: Introduce outliers (Quantity & UnitPrice, 1%) ----
def introduce_outliers_large(df, cols=["Quantity","UnitPrice"], frac=0.01, mag=10, seed=2):
    np.random.seed(seed)
    df2 = df.copy()
    for col in cols:
        if col not in df2.columns: 
            continue
        idx = np.random.choice(df2.index, int(len(df2)*frac), replace=False)
        df2.loc[idx, col] = df2.loc[idx, col] * mag
    return df2

df_dirty = introduce_outliers_large(df_dirty)
print("Added outliers ✅")

print("Final dirty shape:", df_dirty.shape)
print("Sample dirty rows:")
display(df_dirty.sample(5))

#Save the dirty dataset
dirty_path = "data/raw/sales_and_customer_insights_dirty.xlsx"
df_dirty.to_excel(dirty_path, index=False)
print("✅ Saved full dirty dataset to:", dirty_path)


Copied raw file to: data/raw/sales_and_customer_insights_copy.xlsx
Available sheets: ['Online Retail']
Shape: (10, 8)
Starting shape: (541909, 8)
Added missing values ✅
Added duplicates ✅
Added weird date formats ✅
Added outliers ✅
Final dirty shape: (547328, 8)
Sample dirty rows:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
119708,546563,22188,BLACK HEART CARD HOLDER,4.0,2011-03-15 10:49:00,3.95,15047.0,United Kingdom
297352,562936,84968E,SET OF 16 VINTAGE BLACK CUTLERY,1.0,2011-08-10 17:12:00,12.75,NaN,United Kingdom
507016,579167,22816,CARD MOTORBIKE SANTA,48.0,2011-11-28 14:06:00,0.42,16800.0,United Kingdom
108777,545568,22722,SET OF 6 SPICE TINS PANTRY DESIGN,1.0,2011-03-03 16:22:00,8.29,NaN,United Kingdom
58205,541221,22075,6 RIBBONS ELEGANT CHRISTMAS,1.0,2011-01-14 14:28:00,4.13,NaN,United Kingdom


✅ Saved full dirty dataset to: data/raw/sales_and_customer_insights_dirty.xlsx
